In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import matplotlib.ticker as mtick
from random import sample

## Current Dataset Cleaning

In [16]:
df = pd.read_excel('SBA_Loan_data.xlsx')

In [19]:
def isNaN(string):
    return string != string

In [20]:
def check_state(state1,state2,state3):
    state_list = [state1,state2,state3]
    for state in state_list:
        if state in ['AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA',
           'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME',
           'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM',
           'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX',
           'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY']:
            continue
        else:
            return False
    return True

In [21]:
def format_subprogram(subprogram):
    if subprogram == 'Sec. 504 - Loan Guarantees - Private Sector Financed':
        return 'Private Sector Financed'
    elif subprogram == 'Sec. 504 - Delta loans, funded 9/26/95':
        return 'Delta loans'
    elif subprogram == 'Sec. 504 - Premier Certified Lender Program':
        return 'Premier Certified Lender Program'
    else:
        return 'Refinance'

In [22]:
def format_NaicsCode_indicator(NaicsCode):
    if not math.isnan(NaicsCode):
        return 0
    else:
        return 1

In [23]:
def format_NaicsCode(NaicsCode):
    if not math.isnan(NaicsCode):
        return str(NaicsCode)[:2]
    else:
        return 0

In [24]:
def format_zipcode(state):
    if not math.isnan(state):
        return str(state)[:3]
    else:
        return 0

In [25]:
def format_enddate(ChargeOffDate,LoanStatus,TermInMonths,ApprovalDate):
    if LoanStatus == 'CHGOFF':
        return ChargeOffDate
    elif LoanStatus == 'PIF':
        date_1 = pd.to_datetime(ApprovalDate) + np.timedelta64(int(TermInMonths), 'M')
        date_2 = pd.to_datetime('2014-01-31')
        return min(date_1,date_2)
    else:
        return '2014-01-31'

In [26]:
def format_status(status):
    if status == 'PIF':
        return 1
    elif status == 'CHGOFF':
        return 2
    elif status == 'EXEMPT':
        return 0

In [27]:
def get_quarter_index(date):
    date = str(date)
    return (int(date[:4])-1990)*4 + int((int(date[5:7])-1)//3) 

In [57]:
def clean(df):
    df = df[df['LoanStatus']!='CANCLD']
    df = df[df['TermInMonths']!=0]
    df = df[df['LoanStatus'].isna()==False]
    df['sub_zipcode'] = df.apply(lambda row : format_zipcode(row['BorrZip']), axis = 1)
    df['check_state'] = df.apply(lambda row : check_state(row['BorrState'],row['CDC_State'],row['ProjectState']), axis = 1)
    df = df[df['check_state']==True]
    df['is_Same_Borr_CDC'] = df['BorrState'] == df['CDC_State']
    df['is_Same_Borr_Project'] = df['BorrState'] == df['ProjectState']
    df['log_amount'] = np.log(df['GrossApproval'])
    df['loan_purpose'] = df.apply(lambda row : format_subprogram(row['subpgmdesc']), axis = 1)
    df['indicator_NaicsCode'] = df.apply(lambda row : format_NaicsCode_indicator(row['NaicsCode']), axis = 1)
    df['sub_NaicsCode'] = df.apply(lambda row : format_NaicsCode(row['NaicsCode']), axis = 1)
    df['end_date'] = df.apply(lambda row:format_enddate(row['ChargeOffDate'],row['LoanStatus'],row['TermInMonths'],row['ApprovalDate']),axis=1)
    df = df.drop(columns=['Program', 'BorrName','BorrStreet','BorrCity','CDC_Name','CDC_Street','CDC_City',
                          'ThirdPartyLender_Name','ThirdPartyLender_City','ThirdPartyLender_State','ThirdPartyDollars',
                          'ApprovalFiscalYear','DeliveryMethod','InitialInterestRate','NaicsDescription','ProjectCounty',
                          'subpgmdesc','NaicsCode','CDC_State','CDC_Zip','check_state','BorrZip'])
    df['ApprovalDate'] = pd.to_datetime(df['ApprovalDate'])
    df['end_date'] = pd.to_datetime(df['end_date'])
    df['ApprovalYear'] = df['ApprovalDate'].dt.year
    df['EndYear'] = df['end_date'].dt.year
    begin = pd.to_datetime('1990-01-01')
    df['start_index'] = df.apply(lambda row:get_quarter_index(row['ApprovalDate']),axis=1)
    df['end_index'] = df.apply(lambda row:get_quarter_index(row['end_date']),axis=1)
    df['start_day'] = round((df['ApprovalDate'] - begin)/np.timedelta64(1, 'D')).astype('int')
    df['end_day'] = round((df['end_date'] - begin)/np.timedelta64(1, 'D')).astype('int')
    df['time'] = df['end_day'] - df['start_day']
    df['status'] = df.apply(lambda row:format_status(row['LoanStatus']),axis=1)
    df = df.reset_index()
    df['id'] = df.index
    df = df.drop(columns=['index'])
    return df

In [58]:
df = clean(df)

## Merge with Additional Data

In [59]:
df_fixed = pd.read_csv('data/FixedVariables.csv')
df_gsp = pd.read_csv('data/GSP_StateLevel.csv')
df_PI = pd.read_csv('data/PersonalIncome_StateLevel.csv')
df_unemploy = pd.read_csv('data/UnemploymentRate_StateLevel.csv')
df_industry = pd.read_csv('data/IndustryGDP_97-14.csv')
df_hpi = pd.read_csv('data/HPI.csv')
df_leverage = pd.read_csv('data/Leverage.csv')

In [60]:
def format_csv(df):
    df = df.drop(columns=['Unnamed: 0','date'])  
    df = df.set_index('index')
    return df

In [61]:
def stack_column(df,name):
    df.columns.name = name
    df = df.stack()
    df.name = 'value'
    df = df.reset_index()
    df = df.rename(columns={"index": "quarter_index"})
    return df

In [63]:
def combine_fixed(df1,df2,left_index):
    df2 = format_csv(df2)
    df2['quarter_index'] = df2.index
    df = df1.merge(df2,left_on = left_index,right_on='quarter_index',how='left')
    df = df.drop(columns=['quarter_index']) 
    return df

In [64]:
def combine_level(df1,df2,level_name,left_feature,feature_name):
    df2 = format_csv(df2)
    df2 = stack_column(df2,level_name)
    df = df1.merge(df2,left_on = ['start_index',left_feature],right_on =['quarter_index',level_name],how='left')
    df = df.drop(columns=[level_name]) 
    df = df.drop(columns=['quarter_index']) 
    df = df.rename(columns={"value": feature_name})
    return df

In [65]:
df_combined = combine_fixed(df,df_fixed[['Unnamed: 0','date','index','S&P500','VIX']],'end_index')
df_combined = combine_fixed(df_combined,df_fixed[['Unnamed: 0','date','index','TED','PRIME']],'start_index')
df_combined = combine_fixed(df_combined,df_leverage,'end_index')
df_combined = combine_level(df_combined,df_gsp,'state','ProjectState','GSP')
df_combined = combine_level(df_combined,df_PI,'state','ProjectState','PersonalIncome')
df_combined = combine_level(df_combined,df_unemploy,'state','ProjectState','UnemploymentRate')
df_combined = combine_level(df_combined,df_industry,'industry','sub_NaicsCode','IndustryGDP')
df_combined = combine_level(df_combined,df_hpi,'zipcode','sub_zipcode','HPI')

In [67]:
def get_fixed_impute(df,name):
    df = df.drop(columns=['Unnamed: 0','date'])  
    df = df[~df[name].isna()]
    df = df.set_index('index')
    return df.iloc[0,0]

In [68]:
def get_level_impute(df):
    df = df.drop(columns=['Unnamed: 0','date'])  
    df = df.set_index('index')
    df = df.dropna()
    df = df.reset_index()
    df = df.drop(columns=['index'])
    sub = dict(df.iloc[0])
    return sub

In [69]:
leverage_impute = get_fixed_impute(df_leverage,'Leverage')
hpi_impute = get_level_impute(df_hpi)
industry_impute = get_level_impute(df_industry)
unemploy_impute = get_level_impute(df_unemploy)
PI_impute = get_level_impute(df_PI)
gsp_impute = get_level_impute(df_gsp)

In [70]:
def format_missing_indicator(feature):
    if not math.isnan(feature):
        return 0
    else:
        return 1

In [71]:
def format_impute(feature,level,impute):
    if not math.isnan(feature):
        return feature
    else:
        if level == None:
            return impute
        else:
            try:
                return impute[str(level)]
            except:
                return min(impute)

In [72]:
df_combined['indicator_Leverage'] = df_combined.apply(lambda row : format_missing_indicator(row['Leverage']), axis = 1)
df_combined['Leverage'] = df_combined.apply(lambda row : format_impute(row['Leverage'],None,leverage_impute), axis = 1)

df_combined['indicator_HPI'] = df_combined.apply(lambda row : format_missing_indicator(row['HPI']), axis = 1)
df_combined['HPI'] = df_combined.apply(lambda row : format_impute(row['HPI'],row['sub_zipcode'],hpi_impute), axis = 1)

df_combined['indicator_IndustryGDP'] = df_combined.apply(lambda row : format_missing_indicator(row['IndustryGDP']), axis = 1)
df_combined['IndustryGDP'] = df_combined.apply(lambda row : format_impute(row['IndustryGDP'],row['sub_NaicsCode'],industry_impute), axis = 1)

df_combined['indicator_UnemploymentRate'] = df_combined.apply(lambda row : format_missing_indicator(row['UnemploymentRate']), axis = 1)
df_combined['UnemploymentRate'] = df_combined.apply(lambda row : format_impute(row['UnemploymentRate'],row['ProjectState'],unemploy_impute), axis = 1)

df_combined['indicator_PersonalIncome'] = df_combined.apply(lambda row : format_missing_indicator(row['PersonalIncome']), axis = 1)
df_combined['PersonalIncome'] = df_combined.apply(lambda row : format_impute(row['PersonalIncome'],row['ProjectState'],PI_impute), axis = 1)

df_combined['indicator_GSP'] = df_combined.apply(lambda row : format_missing_indicator(row['GSP']), axis = 1)
df_combined['GSP'] = df_combined.apply(lambda row : format_impute(row['GSP'],row['ProjectState'],gsp_impute), axis = 1)


In [73]:
id_list = list(df_combined['id'][df_combined['ApprovalYear']>=2010].unique())

In [74]:
portfolios_list = sample(id_list,500)

In [75]:
def format_sample(loan_id,portfolios_list):
    if loan_id in portfolios_list:
        return 1
    else:
        return 0

In [76]:
df_combined_1 = df_combined.copy()

In [77]:
df_combined_1['sample'] = df_combined.apply(lambda row:format_sample(row['id'],portfolios_list),axis=1)

In [78]:
df_portfolios = df_combined_1[df_combined_1['sample']==1]
df_train = df_combined_1[df_combined_1['sample']==0]

In [79]:
df_portfolios = df_portfolios.drop(columns=['sample'])
df_train = df_train.drop(columns=['sample'])

In [80]:
df_train.to_csv('time_independent_training_dataset.csv')

In [81]:
df_portfolios.to_csv('time_independent_portfolio_500_loans.csv')

In [82]:
df_combined.to_csv('time_independent_combined_dataset.csv')